# **양자과학기술 저자명 통일**

- column 'matched'로 표시된 row는 정확한 정보를 의미함
- WoS에서 절반밖에 제공하지 않음
- affiliation 정보가 있는 저자만 정확한 정보라고 볼 수 있음 

In [21]:
import pandas as pd
import numpy as np
from collections import Counter

dir = 'D:/GD_awekimm/[YU]/[Project]/[Quantum]/Quantum_2nd/04_Analysis/QuanTech_R2/R file/'

# 1. 데이터 로딩 (기존과 동일)
df = pd.read_csv(dir+'quant_author_ed_eu_val.csv')
inst = pd.read_csv(dir+'quant_inst_ed_eu_val_cleaned.csv')

required_columns = [
    'pubid', 'city', 'country', 'pubyear', 'author_id_te', 
    'full_name', 'matched', 'organization_cleaned', 'suborganization_cleaned'
]
df = df[required_columns]
inst = inst[['pubid', 'organization_cleaned', 'suborganization_cleaned']]

df = df.drop_duplicates()
df['full_name'] = df['full_name'].str.replace(r'\.$', '', regex=True)

# 2. '정답' 그룹과 '처리 대상' 그룹 분리 (기존과 동일)
df_matched = df[df['matched'] == 'matched'].copy()
df_rest = df[df['matched'] == 'rest'].copy()

# =================================================================
# ⭐️ 3. (신규) 'rest' 데이터 소속 기관 추론 로직 ⭐️
# =================================================================

# 3-1. (준비) 추론에 필요한 정보 생성
# 'matched' 데이터에서 기관-위치 정보 맵 생성
inst_location_map = df_matched.groupby('organization_cleaned')[['city', 'country']].first().to_dict('index')
# 'matched' 데이터에서 논문별(pubid) 소속 기관 정보 생성
pub_org_map = df_matched.groupby('pubid')['organization_cleaned'].unique().apply(list).to_dict()

# 3-2. (증거 수집) 각 'rest' 레코드에 대한 후보 기관 수집
candidate_orgs = []
for idx, row in df_rest.iterrows():
    pubid = row['pubid']
    city = row['city']
    country = row['country']
    
    candidates = []
    
    # [증거 1: 공동 저자] 같은 논문의 'matched' 저자 소속 기관을 후보로 추가
    if pubid in pub_org_map:
        candidates.extend(pub_org_map[pubid])
        
    # [증거 2: 지리 정보] 해당 논문의 모든 연관 기관 중, 위치가 일치하는 기관을 후보로 추가
    # inst 테이블에는 한 pubid에 여러 기관이 있을 수 있음
    possible_orgs = inst[inst['pubid'] == pubid]['organization_cleaned'].unique()
    for org in possible_orgs:
        if org in inst_location_map:
            loc = inst_location_map[org]
            if loc['city'] == city and loc['country'] == country:
                candidates.append(org)

    candidate_orgs.append(candidates)

df_rest['candidate_orgs'] = candidate_orgs

# --- 하이퍼파라미터 설정 ---
# '핵심 저자'로 판단할 최소 논문 수
FREQUENCY_THRESHOLD = 5
# '핵심 저자'의 기관을 결정하기 위한 최소 신뢰도(증거 일치율)
HIGH_CONFIDENCE_THRESHOLD = 0.8  # 80%

# 3-3-1. 저자별 논문 수를 계산하여 '핵심 저자' 그룹을 정의
author_counts = df['full_name'].value_counts()
frequent_authors = set(author_counts[author_counts >= FREQUENCY_THRESHOLD].index)

# 3-3-2. 저자 프로필 생성 (기존과 동일)
author_profiles = {}
# 'matched' 데이터로 프로필 초기화
for _, row in df_matched.iterrows():
    author = row['full_name']
    org = row['organization_cleaned']
    if author not in author_profiles:
        author_profiles[author] = []
    author_profiles[author].append(org)

# 'rest' 데이터의 후보 기관들을 프로필에 추가
for _, row in df_rest.iterrows():
    author = row['full_name']
    if author not in author_profiles:
        author_profiles[author] = []
    author_profiles[author].extend(row['candidate_orgs'])

# 3-3-3. 저자별로 중요도에 따라 다르게 'inferred_org' 결정
inferred_org_map = {}
for author, org_list in author_profiles.items():
    if not org_list:
        continue

    counts = Counter(org_list)
    most_common_org, top_count = counts.most_common(1)[0]
    
    # [Safe Mode] 저자가 '핵심 저자' 그룹에 속하는 경우
    if author in frequent_authors:
        total_evidence = len(org_list)
        confidence = top_count / total_evidence
        
        # 신뢰도가 설정된 임계값을 넘을 때만 기관을 할당
        if confidence >= HIGH_CONFIDENCE_THRESHOLD:
            inferred_org_map[author] = most_common_org
        # 넘지 못하면 실수를 피하기 위해 할당하지 않음 (결과적으로 NaN)
            
    # [Standard Mode] 저자가 '일반 저자' 그룹에 속하는 경우
    else:
        # 기존 방식대로 가장 빈도가 높은 기관을 할당
        inferred_org_map[author] = most_common_org

# 3-4-1. 추론된 주 기관명을 먼저 할당합니다.
df_rest['organization_cleaned'] = df_rest['full_name'].map(inferred_org_map)

# 3-4-2. 기관명에 맞는 city, country를 매핑하기 위한 map을 준비합니다.
# inst_location_map에서 city와 country를 위한 별도의 딕셔너리를 생성합니다.
city_map = {org: loc['city'] for org, loc in inst_location_map.items()}
country_map = {org: loc['country'] for org, loc in inst_location_map.items()}

# 3-4-3. city와 country 정보를 업데이트합니다.
# 추론된 organization_cleaned를 기준으로 새로운 위치 정보를 매핑합니다.
# 만약 매핑되는 정보가 없다면(NaN), 기존의 위치 정보를 그대로 사용(fillna)합니다.
df_rest['city'] = df_rest['organization_cleaned'].map(city_map).fillna(df_rest['city'])
df_rest['country'] = df_rest['organization_cleaned'].map(country_map).fillna(df_rest['country'])

# 3-4-4. suborganization은 여전히 알 수 없으므로 NaN 처리합니다.
df_rest['suborganization_cleaned'] = np.nan 

# 추론에 사용된 임시 컬럼 삭제
df_rest = df_rest.drop(columns=['candidate_orgs'])


# 4. 보호된 그룹과 처리된 그룹 다시 결합 (기존과 유사)
# df_rest에서 기관 정보가 없는(추론 실패한) 경우는 제외하거나 포함할 수 있음. 여기서는 포함.
df_final = pd.concat([df_matched, df_rest], ignore_index=True)


# 5. ID 통일 로직 실행 (기존과 동일)
# ... (이하 ID 통일 및 저장 코드는 모두 동일합니다) ...

# Step 1. 그룹별 canonical ID 찾기
min_ids = df_final.groupby(['full_name', 'organization_cleaned'])['author_id_te'].min()
matched_ids = df_final[df_final['matched'] == 'matched'].groupby(['full_name', 'organization_cleaned'])['author_id_te'].first()
canonical_s = matched_ids.combine_first(min_ids)
canonical_ids = canonical_s.reset_index(name='canonical_id')
canonical_ids['canonical_id'] = canonical_ids['canonical_id'].astype(df_final['author_id_te'].dtype)

# Step 2. df에 canonical ID 병합
df_final = df_final.merge(canonical_ids, on=['full_name','organization_cleaned'], how='left')

# Step 3. 'matched' ID를 최우선으로 하여 ID 전파
final_id_from_matched = df_final[df_final['matched'] == 'matched'].groupby('full_name')['canonical_id'].first()
default_final_id = df_final.groupby('full_name')['canonical_id'].min()
final_id_map = final_id_from_matched.combine_first(default_final_id)
df_final['author_id_te_cleaned'] = df_final['full_name'].map(final_id_map)

# Step 4. cleaned 여부 표시
df_final['cleaned_or_not'] = np.where(
    df_final['author_id_te'] == df_final['author_id_te_cleaned'],
    'original',
    'cleaned'
)

# 6. 최종 정리 및 저장 (기존과 동일)
df_final = df_final.drop(columns=['canonical_id'])
df_final = df_final.drop_duplicates(subset=['author_id_te_cleaned', 'pubid', 'organization_cleaned'])

df_final.to_parquet(dir + 'quant_author_ed_eu_val_cleaned_inferred.parquet')

print("데이터 정제 및 저장이 완료되었습니다.")
print("추론 후 데이터 샘플:")
print(df_final[df_final['matched'] == 'rest'].head())

데이터 정제 및 저장이 완료되었습니다.
추론 후 데이터 샘플:
          pubid     city country  pubyear  author_id_te       full_name  \
367922  52091.0     ROME   ITALY     1998     6290927.0  Castellano, MG   
367923  52091.0     ROME   ITALY     1998    39921397.0     Torrioli, G   
367924  52091.0     ROME   ITALY     1998    22284443.0        Leoni, R   
367925  62208.0  FUKUOKA   JAPAN     1998     9913402.0        Drung, D   
367926  62208.0  FUKUOKA   JAPAN     1998    23906103.0       Ludwig, F   

       matched organization_cleaned suborganization_cleaned  \
367922    rest                  NaN                     NaN   
367923    rest                  NaN                     NaN   
367924    rest                  NaN                     NaN   
367925    rest                  NaN                     NaN   
367926    rest                  NaN                     NaN   

        author_id_te_cleaned cleaned_or_not  
367922             6290927.0       original  
367923            39921400.0        cleaned

In [16]:
import pandas as pd
import numpy as np

dir = 'D:/GD_awekimm/[YU]/[Project]/[Quantum]/Quantum_2nd/04_Analysis/QuanTech_R2/R file/'

# 1. 데이터 로딩
df = pd.read_csv(dir+'quant_author_ed_eu_val.csv')
inst = pd.read_csv(dir+'quant_inst_ed_eu_val_cleaned.csv')

# ⭐️ (핵심) 원본 파일에 '정답' 기관 정보가 반드시 포함되어 있어야 합니다.
# 만약 실제 컬럼 이름이 다르다면 이 부분을 꼭 수정해주세요.
required_columns = [
    'pubid', 'city', 'country', 'pubyear', 'author_id_te', 
    'full_name', 'matched', 'organization_cleaned', 'suborganization_cleaned'
]
df = df[required_columns]
inst = inst[['pubid', 'organization_cleaned', 'suborganization_cleaned']]

df = df.drop_duplicates()
df['full_name'] = df['full_name'].str.replace(r'\.$', '', regex=True)


# 2. '정답' 그룹과 '처리 대상' 그룹 분리
df_matched = df[df['matched'] == 'matched'].copy()
df_rest = df[df['matched'] == 'rest'].copy()


# 3. 각 그룹 처리
# 3-1. (보호) 'matched' 그룹은 '정답'이므로 아무것도 하지 않고 그대로 사용합니다.
#       inst 파일과 병합하지 않으므로 원본 정보가 100% 보존됩니다.

# 3-2. (처리) 'rest' 그룹은 기관 정보가 불확실하므로 inst 파일과 병합합니다.
#       병합 전, 기존의 (불확실한) 기관 컬럼을 삭제하여 충돌을 방지합니다.
df_rest = df_rest.drop(columns=['organization_cleaned', 'suborganization_cleaned'])
df_rest = pd.merge(df_rest, inst, on='pubid', how='left')
del inst


# 4. 보호된 그룹과 처리된 그룹 다시 결합
df_final = pd.concat([df_matched, df_rest], ignore_index=True)


# 5. ID 통일 로직 실행 (이제 안전하게 조합된 데이터를 대상으로 실행)
# Step 1. 그룹별 canonical ID 찾기
min_ids = df_final.groupby(['full_name', 'organization_cleaned'])['author_id_te'].min()
matched_ids = df_final[df_final['matched'] == 'matched'].groupby(['full_name', 'organization_cleaned'])['author_id_te'].first()
canonical_s = matched_ids.combine_first(min_ids)
canonical_ids = canonical_s.reset_index(name='canonical_id')
canonical_ids['canonical_id'] = canonical_ids['canonical_id'].astype(df_final['author_id_te'].dtype)

# Step 2. df에 canonical ID 병합
df_final = df_final.merge(canonical_ids, on=['full_name','organization_cleaned'], how='left')

# Step 3. 'matched' ID를 최우선으로 하여 ID 전파
final_id_from_matched = df_final[df_final['matched'] == 'matched'].groupby('full_name')['canonical_id'].first()
default_final_id = df_final.groupby('full_name')['canonical_id'].min()
final_id_map = final_id_from_matched.combine_first(default_final_id)
df_final['author_id_te_cleaned'] = df_final['full_name'].map(final_id_map)

# Step 4. cleaned 여부 표시
df_final['cleaned_or_not'] = np.where(
    df_final['author_id_te'] == df_final['author_id_te_cleaned'],
    'original',
    'cleaned'
)

# 6. 최종 정리 및 저장
df_final = df_final.drop(columns=['canonical_id'])
# 중복 제거 기준을 더 명확히 하여 데이터 안정성 확보
df_final = df_final.drop_duplicates(subset=['author_id_te_cleaned', 'pubid', 'organization_cleaned'])

# 경로에 특수문자('[', ']')가 없는지 다시 한번 확인해주세요.
df_final.to_parquet(dir + 'quant_author_ed_eu_val_cleaned.parquet')

print("데이터 정제 및 저장이 완료되었습니다.")
print(df_final.head())

데이터 정제 및 저장이 완료되었습니다.
      pubid     city  country  pubyear  author_id_te   full_name  matched  \
0   52091.0  LAQUILA    ITALY     1998     5945201.0  Carelli, P  matched   
1   62208.0   BERLIN  GERMANY     1998     3941773.0    Beyer, J  matched   
2  114324.0     JENA  GERMANY     1998    35695931.0  Schmidl, F  matched   
3  260373.0   JULICH  GERMANY     1998     6902516.0   Chesca, B  matched   
4  273661.0   JULICH  GERMANY     1998     7320318.0    Chong, Y  matched   

                          organization_cleaned   suborganization_cleaned  \
0                         UNIVERSITY OF AQUILA      DIPARTIMENTO ENERGET   
1  PHYSIKALISCH-TECHNISCHE BUNDESANSTALT (PTB)   PHYS TECH BUNDESANSTALT   
2        FRIEDRICH SCHILLER UNIVERSITY OF JENA       INST FESTKORPERPHYS   
3                     FORSCHUNGSZENTRUM JULICH  INST SCHICHT & IONENTECH   
4                     FORSCHUNGSZENTRUM JULICH  INST SCHICHT & IONENTECH   

   author_id_te_cleaned cleaned_or_not  
0             594

In [18]:
import pandas as pd
import numpy as np

dir = 'D:/GD_awekimm/[YU]/[Project]/[Quantum]/Quantum_2nd/04_Analysis/QuanTech_R2/R file/'

# 1, 2, 3단계는 이전과 완전히 동일합니다.
# 1. 데이터 로딩 및 전처리
df = pd.read_csv(dir+'quant_author_ed_eu_val.csv')
inst = pd.read_csv(dir+'quant_inst_ed_eu_val_cleaned.csv')

required_columns = [
    'pubid', 'city', 'country', 'pubyear', 'author_id_te', 
    'full_name', 'matched', 'organization_cleaned', 'suborganization_cleaned'
]
df = df[required_columns]
inst = inst[['pubid', 'organization_cleaned', 'suborganization_cleaned']]

df = df.drop_duplicates()
df['full_name'] = df['full_name'].str.replace(r'\.$', '', regex=True)

# 2. 'matched'와 'rest' 데이터 분리 및 준비
df_matched = df[df['matched'] == 'matched'].copy()
df_rest = df[df['matched'] == 'rest'].copy()

df_rest = df_rest.drop(columns=['organization_cleaned', 'suborganization_cleaned'])
df_rest = pd.merge(df_rest, inst, on='pubid', how='left')

df_final = pd.concat([df_matched, df_rest], ignore_index=True)

# 3. ID 통일 로직 실행
min_ids = df_final.groupby(['full_name', 'organization_cleaned'])['author_id_te'].min()
matched_ids = df_final[df_final['matched'] == 'matched'].groupby(['full_name', 'organization_cleaned'])['author_id_te'].first()
canonical_s = matched_ids.combine_first(min_ids)
canonical_ids = canonical_s.reset_index(name='canonical_id')
canonical_ids['canonical_id'] = canonical_ids['canonical_id'].astype(df_final['author_id_te'].dtype)

df_final = df_final.merge(canonical_ids, on=['full_name','organization_cleaned'], how='left')

final_id_from_matched = df_final[df_final['matched'] == 'matched'].groupby('full_name')['canonical_id'].first()
default_final_id = df_final.groupby('full_name')['canonical_id'].min()
final_id_map = final_id_from_matched.combine_first(default_final_id)
df_final['author_id_te_cleaned'] = df_final['full_name'].map(final_id_map)

df_final['cleaned_or_not'] = np.where(
    df_final['author_id_te'] == df_final['author_id_te_cleaned'], 'original', 'cleaned'
)
df_final = df_final.drop(columns=['canonical_id'])


# --- ⭐️ 여기서부터 수정된 매칭 로직 시작 ⭐️ ---

print("ID 통일 완료. 'rest' 저자 매칭을 시작합니다...")

# 4. 'rest' 저자 매칭을 위한 준비
# 4-1. 저자 프로필 생성 (기존과 동일)
author_history = df_final.groupby('author_id_te_cleaned')['organization_cleaned'].unique().apply(list)

# 4-2. 논문별 참여 기관 목록 생성 (기존과 동일)
pub_orgs = df_final.groupby('pubid')['organization_cleaned'].unique().apply(list)

# ⭐️ (규칙 추가) 4-3. 논문별 '제외할 기관' 맵 생성
# matched 저자의 소속 기관은 다른 rest 저자들의 후보에서 제외합니다.
forbidden_orgs_map = df_final[df_final['matched'] == 'matched'].groupby('pubid')['organization_cleaned'].first()

# 4-4. 처리할 'rest' 데이터 분리 (기존과 동일)
final_matched = df_final[df_final['matched'] == 'matched'].copy()
final_rest = df_final[df_final['matched'] == 'rest'].copy()


# 5. 'rest' 저자 매칭 시도
rest_unique_authors = final_rest.drop_duplicates(subset=['pubid', 'author_id_te_cleaned']).copy()

# ⭐️ (규칙 추가) 매칭 함수에 '제외 규칙' 로직 추가
def find_likely_org_with_rule(row, author_history_map, pub_orgs_map, forbidden_map):
    author_id = row['author_id_te_cleaned']
    pub_id = row['pubid']
    
    history_orgs = author_history_map.get(author_id, [])
    # 원본 리스트 수정을 방지하기 위해 .copy() 사용
    candidate_orgs = pub_orgs_map.get(pub_id, []).copy()
    
    # 새로운 규칙 적용: 이 논문의 '제외할 기관'이 있다면 후보군에서 제거
    forbidden_org = forbidden_map.get(pub_id)
    if forbidden_org and forbidden_org in candidate_orgs:
        candidate_orgs.remove(forbidden_org)
            
    # 필터링된 후보군과 저자 이력을 비교
    intersection = [org for org in candidate_orgs if org in history_orgs]
    
    if len(intersection) == 1:
        return intersection[0]
    else:
        return np.nan

# 수정된 매칭 함수를 적용
rest_unique_authors['likely_organization'] = rest_unique_authors.apply(
    lambda row: find_likely_org_with_rule(row, author_history, pub_orgs, forbidden_orgs_map), axis=1
)

# 6. 결과 결합 (이전과 동일)
rest_matched_success = rest_unique_authors.dropna(subset=['likely_organization'])
rest_matched_success['organization_cleaned'] = rest_matched_success['likely_organization']
rest_matched_success = rest_matched_success[final_matched.columns] 

failed_keys = rest_unique_authors[rest_unique_authors['likely_organization'].isna()][['pubid', 'author_id_te_cleaned']]
rest_still_unmatched = pd.merge(final_rest, failed_keys, on=['pubid', 'author_id_te_cleaned'])

final_combined = pd.concat([final_matched, rest_matched_success, rest_still_unmatched], ignore_index=True)

# 7. 최종 저장 (이전과 동일)
final_combined = final_combined.drop_duplicates()
final_combined.to_parquet(dir + 'quant_author_ed_eu_val_final_matched.parquet')


print("모든 작업 완료. 최종 데이터가 저장되었습니다.")
matched_count = len(rest_matched_success)
total_rest_authors = len(rest_unique_authors)
print(f"총 {total_rest_authors}개의 'rest' 저자-논문 조합 중 {matched_count}개를 새로운 규칙에 따라 매칭 성공!")

ID 통일 완료. 'rest' 저자 매칭을 시작합니다...


C:\Users\User\AppData\Local\Temp\ipykernel_16596\724584448.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_matched_success['organization_cleaned'] = rest_matched_success['likely_organization']


모든 작업 완료. 최종 데이터가 저장되었습니다.
총 41547개의 'rest' 저자-논문 조합 중 15492개를 새로운 규칙에 따라 매칭 성공!


**교수님 피드백**
- 전체를 정교화하는게 아니라, 중요한 저자의 식별이 중요함 
- 분야별 논문 수 상위 10명 매뉴얼로 확인

In [1]:
import pandas as pd 
df = pd.read_csv('quant_author_ed_eu_val.csv')
df

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,ID_all,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched
0,1,52091.0,qc111,1,1.0,UNIV AQUILA,DIPARTIMENTO ENERGET,LAQUILA,ITALY,1998,52091-qc111-1-1,author,5945201.0,"Carelli, P","Carelli, P","Carelli, P",P,Carelli,matched
1,2,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,52091-qc111-2-NA,author,5945201.0,"Carelli, P","Carelli, P","Carelli, P",P,Carelli,random
2,3,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,52091-qc111-2-NA,author,6290927.0,"Castellano, MG","Castellano, MG","Castellano, MG",MG,Castellano,random
3,4,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,52091-qc111-2-NA,author,39921397.0,"Torrioli, G","Torrioli, G","Torrioli, G",G,Torrioli,random
4,5,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,52091-qc111-2-NA,author,22284443.0,"Leoni, R","Leoni, R","Leoni, R",R,Leoni,random
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350989,1350990,46599931.0,qc43,3,2.0,INFN,SEZ CATANIA,CATANIA,ITALY,2021,46599931-qc43-3-2,author,13492143.0,"Giannelli, Luigi","Giannelli, Luigi","Giannelli, L",Luigi,Giannelli,matched
1350990,1350991,46599931.0,qc43,4,3.0,UNIV COPENHAGEN,CTR HYBRID QUANTUM NETWORKS,COPENHAGEN,DENMARK,2021,46599931-qc43-4-3,author,37550262.0,"Sorensen, Anders S.","Sorensen, Anders S.","Sorensen, AS",Anders S.,Sorensen,matched
1350991,1350992,46599931.0,qc43,4,3.0,UNIV COPENHAGEN,NIELS BOHR INST,COPENHAGEN,DENMARK,2021,46599931-qc43-4-3,author,37550262.0,"Sorensen, Anders S.","Sorensen, Anders S.","Sorensen, AS",Anders S.,Sorensen,matched
1350992,1350993,46599931.0,qc43,4,3.0,NIELS BOHR INSTITUTE,CTR HYBRID QUANTUM NETWORKS,COPENHAGEN,DENMARK,2021,46599931-qc43-4-3,author,37550262.0,"Sorensen, Anders S.","Sorensen, Anders S.","Sorensen, AS",Anders S.,Sorensen,matched


In [10]:
# group by author_id_te and qc_category and count the number of unique pubid 
df_grouped = df.groupby(['author_id_te', 'qc_category']).agg({'pubid': 'nunique'}).reset_index()
df_grouped.rename(columns={'pubid': 'pub_count'}, inplace=True)
df_grouped

,author_id_te,qc_category,pub_count
0,79.0,qc41,1
1,761.0,qc43,1
2,764.0,qc224,1
3,853.0,qc133,1
4,853.0,qc134,1
...,...,...,...
334947,46522883.0,qc422,1
334948,46523096.0,qc224,1
334949,46523852.0,qc224,1
334950,46523976.0,qc43,1


In [11]:
df_grouped.sort_values(by='pub_count', ascending=False, inplace=True)
df_grouped.head(100)

,author_id_te,qc_category,pub_count
268380,36684500.0,qc211,66
330306,45827585.0,qc143,52
268386,36684500.0,qc224,51
305801,41795374.0,qc422,45
179715,24994194.0,qc143,43
...,...,...,...
131224,17954312.0,qc43,24
238182,32993380.0,qc224,24
126095,17192650.0,qc224,24
126761,17294970.0,qc422,24


In [12]:
df_grouped.describe()

,author_id_te,pub_count
count,3.349520e+05,334952.000000
mean,2.309613e+07,1.458967
std,1.318873e+07,1.417166
min,7.900000e+01,1.000000
25%,1.174916e+07,1.000000
50%,2.291299e+07,1.000000
75%,3.475829e+07,1.000000
max,4.652404e+07,66.000000


In [2]:
# check number of unique author_id_te 
df['author_id_te'].nunique()

193678

In [3]:
# check rows where the same 'author_id_te' has different 'wos_standard' 

df[df.duplicated(subset=['author_id_te'], keep=False) 
   & ~df.duplicated(subset=['author_id_te', 'wos_standard'], keep=False)]

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,ID_all,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched


In [4]:
# check rows where the same 'wos_standard' has different 'author_id_te'

df[df.duplicated(subset=['wos_standard'], keep=False) 
   & ~df.duplicated(subset=['wos_standard', 'author_id_te'], keep=False)]

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,ID_all,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched
1672,1673,3602978.0,qc111,2,NaN,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2000,3602978-qc111-2-NA,author,12991795.0,"Garcia, D","Garcia, D","Garcia, D",D,Garcia,random
1674,1675,3602978.0,qc111,2,NaN,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2000,3602978-qc111-2-NA,author,1963866.0,"Asenjo, A","Asenjo, A","Asenjo, A",A,Asenjo,random
1676,1677,3602978.0,qc111,2,NaN,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2000,3602978-qc111-2-NA,author,25200187.0,"Mandal, K","Mandal, K","Mandal, K",K,Mandal,random
1679,1680,3602978.0,qc111,2,NaN,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2000,3602978-qc111-2-NA,author,15980894.0,"Hernando, A","Hernando, A","Hernando, A",A,Hernando,random
3218,3219,5739314.0,qc111,2,NaN,TU WIEN,INST THEORET PHYS,VIENNA,AUSTRIA,2003,5739314-qc111-2-NA,author,36841768.0,"Sjoqvist, E","Sjoqvist, E","Sjoqvist, E",E,Sjoqvist,random
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350541,1350542,46564882.0,qc43,1,1.0,ETH ZURICH,INST QUANTUM ELECT,ZURICH,SWITZERLAND,2021,46564882-qc43-1-1,author,11651473.0,"Ferri, Francesco","Ferri, Francesco","Ferri, F",Francesco,Ferri,matched
1350543,1350544,46564882.0,qc43,1,3.0,ETH ZURICH,INST QUANTUM ELECT,ZURICH,SWITZERLAND,2021,46564882-qc43-1-3,author,11768669.0,"Finger, Fabian","Finger, Fabian","Finger, F",Fabian,Finger,matched
1350647,1350648,46575304.0,qc43,1,1.0,UNIV COLL DUBLIN,SCH PHYS,DUBLIN,IRELAND,2021,46575304-qc43-1-1,author,19833043.0,"Kiely, Anthony","Kiely, Anthony","Kiely, A",Anthony,Kiely,matched
1350811,1350812,46577964.0,qc43,2,4.0,WAGENINGEN UNIV & RES,HORT & PROD PHYSIOL,WAGENINGEN,NETHERLANDS,2021,46577964-qc43-2-4,author,18958659.0,"Kaiser, Elias","Kaiser, Elias","Kaiser, E",Elias,Kaiser,matched


In [5]:
df[df['wos_standard'] == 'Asenjo, A']

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,ID_all,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched
1674,1675,3602978.0,qc111,2,NaN,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2000,3602978-qc111-2-NA,author,1963866.0,"Asenjo, A","Asenjo, A","Asenjo, A",A,Asenjo,random
1008198,1008199,10948659.0,qc422,1,1.0,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2006,10948659-qc422-1-1,author,1963902.0,"Asenjo, A.","Asenjo, A.","Asenjo, A",A.,Asenjo,matched


**국가, 도시, 기관명, 저자명은 같은데, author_id_te가 다른 경우가 있다.**

# **Author ID Merging Rule**
- 'organization_cleaned'와 'full_name'가 같은 경우, 'author_id_te'를 통합

In [27]:
df['full_name'] = df['full_name'].str.replace(r'\.$', '', regex=True)

In [28]:
df[df['wos_standard'] == 'Asenjo, A']

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,ID_all,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched
1674,1675,3602978.0,qc111,2,NaN,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2000,3602978-qc111-2-NA,author,1963866.0,"Asenjo, A","Asenjo, A","Asenjo, A",A,Asenjo,random
1008198,1008199,10948659.0,qc422,1,1.0,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2006,10948659-qc422-1-1,author,1963902.0,"Asenjo, A.","Asenjo, A","Asenjo, A",A.,Asenjo,matched


In [29]:
# If organization_cleaned and full_name are identical but author_id_te is different, merge them with one author_id_te in a new column 'author_id_te_cleaned'
df['author_id_te_cleaned'] = (
    df.groupby(['organization_cleaned', 'full_name'])['author_id_te']
      .transform('first')
)

In [30]:
df['cleaned_or_not'] = (df['author_id_te_cleaned'] != df['author_id_te']).astype(int)
df.head()

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,...,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched,author_id_te_cleaned,cleaned_or_not
0,1,52091.0,qc111,1,1.0,UNIV AQUILA,DIPARTIMENTO ENERGET,LAQUILA,ITALY,1998,...,author,5945201.0,"Carelli, P","Carelli, P","Carelli, P",P,Carelli,matched,5945201.0,0
1,2,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,...,author,5945201.0,"Carelli, P","Carelli, P","Carelli, P",P,Carelli,random,5945201.0,0
2,3,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,...,author,6290927.0,"Castellano, MG","Castellano, MG","Castellano, MG",MG,Castellano,random,6290927.0,0
3,4,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,...,author,39921397.0,"Torrioli, G","Torrioli, G","Torrioli, G",G,Torrioli,random,39921397.0,0
4,5,52091.0,qc111,2,NaN,CNR,IST ELETTR STATO SOLIDO,ROME,ITALY,1998,...,author,22284443.0,"Leoni, R","Leoni, R","Leoni, R",R,Leoni,random,22284443.0,0


In [34]:
# check number of unique author_id_te 
df['author_id_te'].nunique()

193678

In [33]:
# check number of unique author_id_te 
df['author_id_te_cleaned'].nunique()

177770

In [31]:
df[df['wos_standard'] == 'Asenjo, A']

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,...,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched,author_id_te_cleaned,cleaned_or_not
1674,1675,3602978.0,qc111,2,NaN,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2000,...,author,1963866.0,"Asenjo, A","Asenjo, A","Asenjo, A",A,Asenjo,random,1963866.0,0
1008198,1008199,10948659.0,qc422,1,1.0,CSIC,INST CIENCIA MAT,MADRID,SPAIN,2006,...,author,1963902.0,"Asenjo, A.","Asenjo, A","Asenjo, A",A.,Asenjo,matched,1963866.0,1


**full_name은 같은데 기관명이 다른 경우?**

In [35]:
# check rows where full_name is identical but organization_cleaned is different 

df[df.duplicated(subset=['full_name'], keep=False) & ~df.duplicated(subset=['organization_cleaned'], keep=False)]

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,...,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched,author_id_te_cleaned,cleaned_or_not
4348,4349,7448624.0,qc111,1,1.0,UNIV GENOA GAP,APPL PHYS GRP,GENEVA,SWITZERLAND,2001,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,matched,39814970.0,0
4349,4350,7448624.0,qc111,2,2.0,UNIV VIENNA UNIVIE,INST EXPT PHYS,VIENNA,AUSTRIA,2001,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
9814,9815,13134142.0,qc111,1,1.0,QUANTUM OPT LAB ELSAGDATAMAT,NaN,GENOA,ITALY,2007,...,author,4835077.0,"Bovino, Fabio Antonio","Bovino, Fabio Antonio","Bovino, FA",Fabio Antonio,Bovino,matched,4835077.0,0
10083,10084,13484216.0,qc111,4,5.0,NATL CTR NANOMAT TECHNOL,NaN,POHANG,SOUTH KOREA,2008,...,author,22198874.0,"Lee, Hu-Jong","Lee, Hu-Jong","Lee, HJ",Hu-Jong,Lee,matched,22198874.0,0
10344,10345,13913831.0,qc111,3,1.0,ACCAD NAZL LINCEI,NaN,ROME,ITALY,2008,...,author,8843932.0,"De Martini, Francesco","De Martini, Francesco","De Martini, F",Francesco,De Martini,matched,8843932.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349436,1349437,46301158.0,qc43,2,5.0,AGR RES LTD,NaN,TROUBSKO,CZECH REPUBLIC,2021,...,author,20134164.0,"Kintl, Antonin","Kintl, Antonin","Kintl, A",Antonin,Kintl,matched,20134164.0,0
1349443,1349444,46303705.0,qc43,4,3.0,CNR IOM IST OFFICINA MOL,NaN,TRIESTE,ITALY,2021,...,author,33746753.0,"Romanato, Filippo","Romanato, Filippo","Romanato, F",Filippo,Romanato,matched,33746753.0,0
1349851,1349852,46379898.0,qc43,5,2.0,HKU UCAS JOINT INST THEORET & COMPUTAT PHYS HONG,NaN,HONG KONG,PEOPLES R CHINA,2021,...,author,44231961.0,"Yao, Wang","Yao, Wang","Yao, W",Wang,Yao,matched,44231961.0,0
1350217,1350218,46501668.0,qc43,2,1.0,UV BOOSTING,NaN,BOULOGNE BILLANCOURT,FRANCE,2021,...,author,22150607.0,"Ledermann, Loic","Ledermann, Loic","Ledermann, L",Loic,Ledermann,matched,22150607.0,0


In [36]:
df[df['full_name'] == 'Tittel, Wolfgang']

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,...,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched,author_id_te_cleaned,cleaned_or_not
4348,4349,7448624.0,qc111,1,1.0,UNIV GENOA GAP,APPL PHYS GRP,GENEVA,SWITZERLAND,2001,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,matched,39814970.0,0
13767,13768,18141708.0,qc111,1,10.0,UNIV CALGARY,INST QUANTUM INFORMAT SCI,CALGARY,CANADA,2011,...,author,39814968.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,matched,39814968.0,0
13774,13775,18141708.0,qc111,2,10.0,UNIV CALGARY,DEPT PHYS & ASTRON,CALGARY,CANADA,2011,...,author,39814968.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,matched,39814968.0,0
15657,15658,20280021.0,qc111,1,10.0,UNIV CALGARY,INST QUANTUM INFORMAT SCI,CALGARY,CANADA,2012,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,matched,39814968.0,1
15664,15665,20280021.0,qc111,2,10.0,UNIV CALGARY,DEPT PHYS & ASTRON,CALGARY,CANADA,2012,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,matched,39814968.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1210831,1210832,11236815.0,qc43,2,NaN,UNIV NICE,PHYS MAT CONDENSEE LAB,NICE,FRANCE,2006,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,random,39814970.0,0
1210838,1210839,11236815.0,qc43,3,NaN,UNIV GENEVA,DEPT PHYS MAT CONDENSEE,GENEVA,SWITZERLAND,2006,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,random,39814970.0,0
1210845,1210846,11236815.0,qc43,4,NaN,UNIV CALGARY,INST QUANTUM INFORMAT SCI,CALGARY,CANADA,2006,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,random,39814968.0,1
1239939,1239940,19831747.0,qc43,3,6.0,UNIV CALGARY,INST QUANTUM INFORMAT SCI,CALGARY,CANADA,2011,...,author,39814970.0,"Tittel, Wolfgang","Tittel, Wolfgang","Tittel, W",Wolfgang,Tittel,matched,39814968.0,1


- Tittel, Wolfgang이 실제로 이직한건지 알 수가 없음
- 확인해본 결과, Tittel, Wolfgang은 UNIV GENEVA에서 박사학위를 받고, UNIV CALGARY에서 교수로 재직 중
- 즉, UNIV GENEVA와 UNIV CALGARY에서 진행한 연구는 동일인물일 가능성이 높음
- 하지만, UNIV NICE는 관련 없음 -> 동명이인

In [40]:
sample = df[df['full_name'] == 'Weihs, Gregor']
sample

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,...,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched,author_id_te_cleaned,cleaned_or_not
4329,4330,7200573.0,qc111,1,2.0,UNIV VIENNA,INST EXPT PHYS,VIENNA,AUSTRIA,2003,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
4349,4350,7448624.0,qc111,2,2.0,UNIV VIENNA UNIVIE,INST EXPT PHYS,VIENNA,AUSTRIA,2001,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
11276,11277,15274405.0,qc111,1,4.0,UNIV WATERLOO,INST QUANTUM COMP,WATERLOO,CANADA,2009,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
11280,11281,15274405.0,qc111,2,4.0,UNIV WATERLOO,DEPT PHYS,WATERLOO,CANADA,2009,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
11282,11283,15274405.0,qc111,4,4.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2009,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140798,1140799,21762254.0,qc423,3,3.0,UNIV WATERLOO,DEPT PHYS & ASTRON,WATERLOO,CANADA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
1185502,1185503,44846533.0,qc423,1,6.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2021,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
1247914,1247915,22611039.0,qc43,1,6.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2013,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0
1346455,1346456,45608699.0,qc43,2,7.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2021,...,author,42547602.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,0


In [42]:
# show rows where author_id_te_cleaned is not 42547602.0
sample[sample['author_id_te'] != 42547602.0]

,Unnamed: 0,pubid,qc_category,addr_num,SEQ_NO,organization_cleaned,suborganization,city,country,pubyear,...,role,author_id_te,display_name,full_name,wos_standard,first_name,last_name,matched,author_id_te_cleaned,cleaned_or_not
16902,16903,21762254.0,qc111,1,3.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
16903,16904,21762254.0,qc111,2,3.0,UNIV WATERLOO,INST QUANTUM COMP,WATERLOO,CANADA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
16904,16905,21762254.0,qc111,3,3.0,UNIV WATERLOO,DEPT PHYS & ASTRON,WATERLOO,CANADA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
26413,26414,34524709.0,qc111,3,6.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2017,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
798592,798593,21762254.0,qc41,1,3.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
798593,798594,21762254.0,qc41,2,3.0,UNIV WATERLOO,INST QUANTUM COMP,WATERLOO,CANADA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
798594,798595,21762254.0,qc41,3,3.0,UNIV WATERLOO,DEPT PHYS & ASTRON,WATERLOO,CANADA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
1140796,1140797,21762254.0,qc423,1,3.0,UNIV INNSBRUCK,INST EXPT PHYS,INNSBRUCK,AUSTRIA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
1140797,1140798,21762254.0,qc423,2,3.0,UNIV WATERLOO,INST QUANTUM COMP,WATERLOO,CANADA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1
1140798,1140799,21762254.0,qc423,3,3.0,UNIV WATERLOO,DEPT PHYS & ASTRON,WATERLOO,CANADA,2012,...,author,42547652.0,"Weihs, Gregor","Weihs, Gregor","Weihs, G",Gregor,Weihs,matched,42547602.0,1


- 소속기관이 바뀌어도 author_id_te는 대부분 동일한 경우
- Weihs, Gregor는 캐나다, 도쿄, 미국에서 교수 재직함 
- 현재는 오스트리아에서 교수 재직 중
- 위 경우, 모두 같은 author_id_te_cleaned로 정제됨 

In [44]:
df.to_csv('quant_author_ed_eu_val_cleaned.csv', index=False)